In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [3]:
def show_image_with_contours(image, contours, title='Image with Contours'):
    image_copy = image.copy()
    cv2.drawContours(image_copy, contours, -1, (0, 255, 0), 2)
    plt.imshow(cv2.cvtColor(image_copy, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.show()

def show_image_with_points(image, points, title='Image with Points', color=(0, 0, 255)):
    image_copy = image.copy()
    for point in points:
        cv2.circle(image_copy, (int(point[0]), int(point[1])), 5, color, -1)
    plt.imshow(cv2.cvtColor(image_copy, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.show()

In [4]:
# Read the image
image = cv2.imread('image.jpg')
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian Blur to smooth the image
blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

# Threshold the image to get a binary image
_, thresholded = cv2.threshold(blurred_image, 127, 255, cv2.THRESH_BINARY)

# Find contours
contours, _ = cv2.findContours(thresholded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Display contours
show_image_with_contours(image, contours, title='Contours')

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
# Function to approximate contours
def approximate_contours(contours, epsilon_factor=0.01):
    approx_contours = []
    for contour in contours:
        epsilon = epsilon_factor * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        approx_contours.append(approx)
    return approx_contours

# Approximate the contours
approx_contours = approximate_contours(contours)

# Display approximated contours
show_image_with_contours(image, approx_contours, title='Approximated Contours')

# Function to detect arcs in the approximated contours
def detect_arcs(contours, min_arc_length=50, circularity_threshold=0.7):
    arcs = []
    for contour in contours:
        for i in range(len(contour)):
            for j in range(i + 2, len(contour)):
                segment = contour[i:j]
                if len(segment) < 5:  # Not enough points to fit an ellipse
                    continue
                
                (x, y), (MA, ma), angle = cv2.fitEllipse(segment)
                ellipse_aspect_ratio = min(MA, ma) / max(MA, ma)
                
                if 0.5 < ellipse_aspect_ratio < 1.0:  # Roughly circular
                    arc_length = cv2.arcLength(segment, False)
                    if arc_length > min_arc_length:
                        circularity = (4 * np.pi * cv2.contourArea(segment)) / (arc_length ** 2)
                        if circularity > circularity_threshold:
                            arcs.append(segment)
    return arcs

# Detect arcs
arcs = detect_arcs(approx_contours)

# Display arcs
arc_points = [point[0] for arc in arcs for point in arc]
show_image_with_points(image, arc_points, title='Detected Arcs')

# Function to calculate tangents
def calculate_tangents(arc):
    (x, y), radius = cv2.minEnclosingCircle(arc)
    center = (int(x), int(y))

    pt1 = tuple(arc[0][0])
    pt2 = tuple(arc[-1][0])

    def tangent_point(pt):
        vec = np.array(pt) - np.array(center)
        tangent_vec = np.array([-vec[1], vec[0]])
        tangent_vec = tangent_vec / np.linalg.norm(tangent_vec) * radius
        tangent_pt = tuple((np.array(center) + tangent_vec).astype(int))
        return tangent_pt

    tangent1 = tangent_point(pt1)
    tangent2 = tangent_point(pt2)

    return (pt1, tangent1), (pt2, tangent2)

# Calculate tangents for all arcs
tangents = []
for arc in arcs:
    tangents.extend(calculate_tangents(arc))

# Function to find line intersection
def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        return None

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x, y

# Find intersections
intersection_points = []
for i in range(0, len(tangents), 2):
    if i + 1 < len(tangents):
        intersect_point = line_intersection(tangents[i], tangents[i + 1])
        if intersect_point:
            intersection_points.append(intersect_point)

# Display tangents
output_image_tangents = image.copy()
for line in tangents:
    cv2.line(output_image_tangents, line[0], line[1], (255, 0, 0), 2)
show_image_with_contours(output_image_tangents, contours, title='Tangents')

# Display intersections
show_image_with_points(image, intersection_points, title='Intersection Points')

# Final result with contours, tangents, and intersection points
final_output = output_image_tangents.copy()
for point in intersection_points:
    cv2.circle(final_output, (int(point[0]), int(point[1])), 5, (0, 0, 255), -1)
plt.imshow(cv2.cvtColor(final_output, cv2.COLOR_BGR2RGB))
plt.title('Final Output with Contours, Tangents, and Intersections')
plt.show()